# Loop 27 Strategic Analysis

## Key Findings:
1. All local search approaches converge to 70.624381
2. Gap to target: 1.705 points (2.41%)
3. Target IS achievable (above theoretical minimum)
4. Need fundamentally different approach

In [ ]:
# Analyze per-N scores to find where improvements are most needed
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate

TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def parse_s_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

def create_tree_polygon(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_bounding_box_side(trees):
    all_x, all_y = [], []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def calculate_score(positions, n):
    trees = [create_tree_polygon(p[0], p[1], p[2]) for p in positions]
    side = get_bounding_box_side(trees)
    return (side ** 2) / n

# Load current best
df = pd.read_csv('/home/submission/submission.csv')
df['x'] = df['x'].apply(parse_s_value)
df['y'] = df['y'].apply(parse_s_value)
df['deg'] = df['deg'].apply(parse_s_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

# Calculate per-N scores
tree = Polygon(TREE_TEMPLATE)
tree_area = tree.area
print(f'Single tree area: {tree_area:.6f}')

scores = {}
for n in range(1, 201):
    group = df[df['n'] == n]
    positions = np.array([[row['x'], row['y'], row['deg']] for _, row in group.iterrows()])
    scores[n] = calculate_score(positions, n)

# Calculate efficiency and improvement potential
print('\n=== Top 20 N values with most improvement potential ===')
efficiency = [(n, scores[n], scores[n] / tree_area, scores[n] - tree_area) for n in range(1, 201)]
efficiency.sort(key=lambda x: -x[3])  # Sort by improvement potential

for n, score, eff, potential in efficiency[:20]:
    print(f'N={n:3d}: score={score:.6f}, efficiency={eff:.3f}x, potential={potential:.6f}')

print(f'\nTotal score: {sum(scores.values()):.6f}')
print(f'Target: 68.919154')
print(f'Gap: {sum(scores.values()) - 68.919154:.6f}')

In [ ]:
# Analyze what percentage of improvement we need per N
total_score = sum(scores.values())
target = 68.919154
gap = total_score - target

print(f'Total gap to close: {gap:.6f}')
print(f'\nIf we improve each N by the same percentage:')
required_pct = gap / total_score * 100
print(f'Required improvement: {required_pct:.2f}%')

print(f'\nIf we focus on N=1-20 (highest potential):')
n1_20_score = sum(scores[n] for n in range(1, 21))
n1_20_potential = sum(scores[n] - tree_area for n in range(1, 21))
print(f'N=1-20 current score: {n1_20_score:.6f}')
print(f'N=1-20 improvement potential: {n1_20_potential:.6f}')
print(f'If we capture 50% of N=1-20 potential: {n1_20_potential * 0.5:.6f} improvement')

print(f'\nIf we focus on N=1-50:')
n1_50_score = sum(scores[n] for n in range(1, 51))
n1_50_potential = sum(scores[n] - tree_area for n in range(1, 51))
print(f'N=1-50 current score: {n1_50_score:.6f}')
print(f'N=1-50 improvement potential: {n1_50_potential:.6f}')
print(f'If we capture 20% of N=1-50 potential: {n1_50_potential * 0.2:.6f} improvement')

In [ ]:
# Check what the theoretical best score would be for different efficiency levels
print('=== Theoretical scores at different efficiency levels ===')
for eff_mult in [1.0, 1.1, 1.2, 1.3, 1.4, 1.5]:
    theoretical = tree_area * eff_mult * 200
    print(f'Efficiency {eff_mult:.1f}x: {theoretical:.6f}')

print(f'\nCurrent average efficiency: {total_score / (tree_area * 200):.3f}x')
print(f'Target average efficiency: {target / (tree_area * 200):.3f}x')

In [ ]:
# Analyze the distribution of angles in the current solution
print('=== Angle distribution analysis ===')
for n in [1, 2, 3, 4, 5, 10, 20, 50, 100]:
    group = df[df['n'] == n]
    angles = group['deg'].values % 360
    unique_angles = np.unique(np.round(angles, 1))
    print(f'N={n}: angles = {unique_angles[:5]}... ({len(unique_angles)} unique)')